<a href="https://colab.research.google.com/github/powerstone666/mlModels/blob/main/knn(Music_Recommendation_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauthamvijayaraj/spotify-tracks-dataset-updated-every-week")

print("Path to dataset files:", path)

100%|██████████| 4.72M/4.72M [00:00<00:00, 58.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gauthamvijayaraj/spotify-tracks-dataset-updated-every-week/versions/2


In [2]:
ls /root/.cache/kagglehub/datasets/gauthamvijayaraj/spotify-tracks-dataset-updated-every-week/versions/2

spotify_tracks.csv


In [186]:
import pandas as pd
import numpy as np
import plotly.express as px
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import re

In [239]:
data=pd.read_csv("/root/.cache/kagglehub/datasets/gauthamvijayaraj/spotify-tracks-dataset-updated-every-week/versions/2/spotify_tracks.csv")
map_lang={lang:i + 1 for i, lang in enumerate(data['language'].unique())}
data['language']=data['language'].map(map_lang)
# Normalize track_name and artist_name columns to avoid case or extra space differences
data['track_name'] = data['track_name'].str.strip().str.lower()
data['artist_name'] = data['artist_name'].str.strip().str.lower()

# Drop duplicates based on normalized track_name and artist_name
data = data.drop_duplicates(subset=['track_name', 'artist_name'])

# Reset index after dropping duplicates (optional)
data = data.reset_index(drop=True)


data.head()


,track_id,track_name,artist_name,year,popularity,artwork_url,album_name,acousticness,danceability,duration_ms,...,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,track_url,language
0,2r0ROhr7pRN4MXDMT1fEmd,"leo das entry (from ""leo"")",anirudh ravichander,2024,59,https://i.scdn.co/image/ab67616d0000b273ce9c65...,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,...,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,https://open.spotify.com/track/2r0ROhr7pRN4MXD...,1
1,4I38e6Dg52a2o2a8i5Q5PW,aao killelle,"anirudh ravichander, pravin mani, vaishali sri...",2024,47,https://i.scdn.co/image/ab67616d0000b273be1b03...,AAO KILLELLE,0.0851,0.780,207369.0,...,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,https://open.spotify.com/track/4I38e6Dg52a2o2a...,1
2,59NoiRhnom3lTeRFaBzOev,mayakiriye sirikiriye - orchestral edm,"anirudh ravichander, anivee, alvin bruno",2024,35,https://i.scdn.co/image/ab67616d0000b27334a1dd...,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,...,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,https://open.spotify.com/track/59NoiRhnom3lTeR...,1
3,5uUqRQd385pvLxC8JX3tXn,scene ah scene ah - experimental edm mix,"anirudh ravichander, bharath sankar, kabilan, ...",2024,24,https://i.scdn.co/image/ab67616d0000b27332e623...,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,...,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,https://open.spotify.com/track/5uUqRQd385pvLxC...,1
4,1KaBRg2xgNeCljmyxBH1mo,gundellonaa x i am a disco dancer - mashup,"anirudh ravichander, benny dayal, leon james, ...",2024,22,https://i.scdn.co/image/ab67616d0000b2735a59b6...,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,...,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,https://open.spotify.com/track/1KaBRg2xgNeCljm...,1


In [240]:
spotifyData=data[["track_name","artist_name","year","popularity","album_name","acousticness","danceability","liveness","loudness","speechiness","tempo","valence","language"]]

In [242]:
features = ["popularity", "acousticness", "danceability", "liveness", "loudness", "speechiness", "tempo", "valence","language"]

scaler = MinMaxScaler()
training_data = scaler.fit_transform(spotifyData[features])


data_scaled = pd.DataFrame(training_data, columns=features)


In [243]:
model=NearestNeighbors(n_neighbors=10,metric="cosine")
model.fit(data_scaled)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [255]:
song_name = "ek villain"
new_name=song_name.strip()
song_index = spotifyData[spotifyData['track_name'].str.contains(re.escape(new_name), case=False, na=False)].index[0]


song_features = data_scaled.iloc[song_index].values.reshape(1, -1)


distances, indices = model.kneighbors(song_features)


print("Original Song:")
print(spotifyData.iloc[song_index][['track_name', 'artist_name']])

recommended_songs = []

for idx in indices[0][1:]:
    recommended_song = spotifyData.iloc[idx][['track_name', 'artist_name']]
    recommended_songs.append(recommended_song)

recommended_songs_df = pd.DataFrame(recommended_songs)


print("\nRecommended Songs:")
recommended_songs_df.head(10)


Original Song:
track_name     humdard (from "ek villain")
artist_name                   arijit singh
Name: 5640, dtype: object

Recommended Songs:


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_name,artist_name
5943,humdard,arijit singh
5993,aasan nahin yahan,arijit singh
32646,gul,anuv jain
5310,"mann jogiya (from ""pyaar hai toh hai"")","arijit singh, ishita vishwakarma, anique"
38589,maana ke hum yaar nahin,"sachin-jigar, parineeti chopra, kausar munir"
10232,"o bedardeya (from ""tu jhoothi main makkaar"")","pritam, arijit singh, amitabh bhattacharya"
5695,"tujhe kitna chahne lage (from ""kabir singh"")","arijit singh, mithoon"
5697,tujhe kitna chahne lage,arijit singh
5698,"thodi jagah (from ""marjaavaan"")","arijit singh, tanishk bagchi"
